# Simulation from Python

You can use the GLASS `Simulation` interface from Python. The process is the same as using the configuration files: Create a simulation, then add functions to it.

In [1]:
from glass.simulation import Simulation

## Setting up the simulation

The simulation is initialised with the options you would find in the `[config]` section of a configuration file.

In [2]:
nside = 512
zbins = [0.1, 0.2, 0.3, 0.4, 0.5]

In [3]:
sim = Simulation(nside=nside, zbins=zbins)

## Adding functions

You can then add functions to your simulation, similar to what you would find in the `[simulation]` section of a configuration file. However, from Python, the functions must be imported first.

In [4]:
from glass.cosmology import cosmology
from glass.cls import cls_from_pyccl
from glass.matter import lognormal_matter
from glass.lensing import lognormal_convergence, shear_from_convergence

You can then add the steps of the simulation. The `add()` function takes the function to call and its arguments. The return value of `add()` is the function call that will be used in the simulation.

In [5]:
sim.add(cosmology, H0=70, Om=0.3)

cosmology = cosmology(H0=70, Om=0.3)

The next step will be calling `cls_from_pyccl` to compute the Cls of the random fields. However, that functions requires the redshift bins and cosmology as arguments. You can see in the output below that the simulation resolves the names by itself.

In [6]:
sim.add(cls_from_pyccl, fields={'matter': None, 'convergence': None}, lmax=1000)

theory_cls = cls_from_pyccl({'matter': None, 'convergence': None}, 1000, zbins, cosmology)

Next are the random fields. These are handled in a special way by the simulator, since all random fields must be simulated in a single step, due to correlations. But this is done internally, and they are added as any other function.

In [7]:
sim.add(lognormal_matter)

matter = lognormal_matter(nbins)

In [8]:
sim.add(lognormal_convergence)

convergence = lognormal_convergence(zbins)

Finally, add a simulation step that is not a random field.

In [9]:
sim.add(shear_from_convergence)

shear = shear_from_convergence(convergence)

## Running the simulation

Once steps have been added, the simulation runs by calling the `run()` method. This returns a `state` dictionary that contains the names and outputs of all functions added above.

In [10]:
state = sim.run()

In [11]:
list(state.keys())

['nside',
 'zbins',
 'nbins',
 'cosmology',
 'theory_cls',
 'matter',
 'convergence',
 'shear',
 '__run__',
 'cls',
 'gaussian_cls',
 'reg_gaussian_cls',
 'regularized_cls']